In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
import os

os.environ["GOOGLE_API_KEY"] = ""
llm = ChatGoogleGenerativeAI(model = "gemini-pro",temperature=0)

In [24]:
from crewai import Crew, Agent, Task


ranker_agent = Agent(
    role="Patient Ranking Agent",
    goal="To rank patients based on the urgency of their condition from 1 to 10.",
    backstory="An analytical and precise virtual assistant designed to prioritize patients based on the severity of their symptoms and conditions based on the patients general data and it's visual data.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# Define the ranking task
rank_patient_task = Task(
    description=(
        "1. Assess the patient's symptoms and conditions baed on the rules: {rules}  ."
        "2. Rank the patient based on the general information: {general_information} and visual information {visual_information}."
        "3. Assign a score from 1 to 10."
    ),
    expected_output="Only a numeric output of  ranked score from 1 to 10 based on the urgency of the patient's condition.",
    agent=ranker_agent,
)


In [30]:
rules = """
Severe but Not Immediately Life-Threatening Conditions:

High Fever for Several Days (e.g., 103°F or higher for more than 2 days): 3 points
Severe Abdominal Pain (potential appendicitis): 3 points
Uncontrolled Bleeding (e.g., deep cuts, heavy nosebleed): 3 points
Severe Dehydration (due to vomiting/diarrhea): 3 points
Moderate Conditions:

Moderate Fever (e.g., 101-102°F): 2 points
Persistent Vomiting or Diarrhea: 2 points
Severe Headache (potential migraine): 2 points
Stomach Ache from Eating Stale Food: 2 points
Mild Conditions:

Mild Cold or Flu Symptoms: 1 point
Mild Allergic Reactions (mild rash, itching): 1 point
Minor Cuts and Scrapes: 1 point
Routine Check-ups and Follow-ups: 1 point
Additional Factors to Consider:
Patient Age and Overall Health:

Infant or Elderly: +2 points
Symptom Duration and Progression:

Symptoms for more than 3 days: +2 points
Effectiveness of Medication:

Medication not having any effect or having effect for very short time: +2 points
Ranking Calculation:
Initialize the score:

Start with 0 points.
Assign points based on the primary condition:

Add points according to the condition category (Severe, Moderate, Mild).
Add points for additional factors:

Add 2 points if the patient is an infant or elderly (age greater than 50 or age less than 8).
Add 2 points if the symptoms have persisted for more than 4 days.
Add 2 points if the medication is not effective.
Determine the urgency level:
"""

In [31]:
general_information = "Patient Name: Sneh Age: 20 Symptoms: - High fever (99 degrees Fahrenheit) - Cold - Cough Pain Assessment: None provided Medication: Paracetamol, effective for a few hours Symptom Duration: 2 days Additional Information: None"

In [32]:
vision_information = """
Skin Color and Condition:

Pale Skin: The patient's skin appears slightly pale, which could indicate anemia, shock, or lack of oxygen.
Yellowish Skin (Jaundice): No signs of yellowing are observed in the skin.
Blue Skin or Lips (Cyanosis): No blue discoloration is visible on the skin or lips.
Redness: There is significant redness on the cheeks and nose, which may suggest a condition like rosacea.
Rashes or Hives: No rashes or hives are visible on the face.
Pimples, Pustules, or Pox: No pimples, pustules, or pox are observed.
Eyes:

Dark Circles: There are dark circles under the eyes, suggesting sleep deprivation, stress, or chronic fatigue.
Yellowing (Scleral Icterus): No yellowing is observed in the sclera.
Red or Bloodshot Eyes: The eyes appear slightly red or bloodshot, which could indicate an infection, allergy, or irritation.
Drooping Eyelids (Ptosis): The eyelids appear slightly droopy, which may be due to fatigue or neurological issues.
Mouth and Lips:

Blue Lips (Cyanosis): The lips appear slightly blue, indicating poor oxygenation.
Swollen Lips: There is no swelling observed on the lips.
Cracked or Dry Lips: The lips appear dry, which may indicate dehydration or vitamin deficiencies.
Neck region:

Swollen Glands: No swollen glands are visible in the neck region.
Visible Thyroid Enlargement (Goiter): The thyroid gland is not visibly enlarged.
"""

In [33]:
crew = Crew(agents=[ranker_agent], tasks=[rank_patient_task],verbose=2)
input_data = {"rules": rules, "general_information": general_information, "visual_information": vision_information}

2024-06-26 17:06:41,068 - 11664 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [34]:
result = crew.kickoff(input_data)

 [2024-06-26 17:06:41][DEBUG]: == Working Agent: Patient Ranking Agent
 [2024-06-26 17:06:41][INFO]: == Starting Task: 1. Assess the patient's symptoms and conditions baed on the rules: 
Severe but Not Immediately Life-Threatening Conditions:

High Fever for Several Days (e.g., 103°F or higher for more than 2 days): 3 points
Severe Abdominal Pain (potential appendicitis): 3 points
Uncontrolled Bleeding (e.g., deep cuts, heavy nosebleed): 3 points
Severe Dehydration (due to vomiting/diarrhea): 3 points
Moderate Conditions:

Moderate Fever (e.g., 101-102°F): 2 points
Persistent Vomiting or Diarrhea: 2 points
Severe Headache (potential migraine): 2 points
Stomach Ache from Eating Stale Food: 2 points
Mild Conditions:

Mild Cold or Flu Symptoms: 1 point
Mild Allergic Reactions (mild rash, itching): 1 point
Minor Cuts and Scrapes: 1 point
Routine Check-ups and Follow-ups: 1 point
Additional Factors to Consider:
Patient Age and Overall Health:

Infant or Elderly: +2 points
Symptom Duration a